In [ ]:
import pandas as pd
import kagglehub

In [2]:
# Only if you wanna download the original dataset and query with sql
# theres also a play_by_play.csv file that was too big to push but ion think we need it
#------------------------------------------------------------------------------------------------
# path = kagglehub.dataset_download("wyattowalsh/basketball")

# print("Path to dataset files:", path)

In [5]:
players = pd.read_csv('../data/player.csv')

In [9]:
active_players = players[players['is_active'] == 1]
active_players

,id,full_name,first_name,last_name,is_active
10,1630173,Precious Achiuwa,Precious,Achiuwa,1
22,203500,Steven Adams,Steven,Adams,1
24,1628389,Bam Adebayo,Bam,Adebayo,1
29,1630534,Ochai Agbaji,Ochai,Agbaji,1
41,1630583,Santi Aldama,Santi,Aldama,1
...,...,...,...,...,...
4806,201152,Thaddeus Young,Thaddeus,Young,1
4808,1629027,Trae Young,Trae,Young,1
4809,1630209,Omer Yurtseven,Omer,Yurtseven,1
4812,203469,Cody Zeller,Cody,Zeller,1


In [13]:
games = pd.read_csv('../data/game.csv')
games.sort_values('game_date').tail(50)

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,season_type
65646,42022,1610612738,BOS,Boston Celtics,42200115,2023-04-25 00:00:00,BOS vs. ATL,L,240,49.0,...,34.0,26.0,5.0,4.0,9.0,16.0,119.0,2,1,Playoffs
65650,42022,1610612739,CLE,Cleveland Cavaliers,42200135,2023-04-26 00:00:00,CLE vs. NYK,L,240,35.0,...,48.0,21.0,6.0,5.0,14.0,17.0,106.0,11,1,Playoffs
65649,42022,1610612763,MEM,Memphis Grizzlies,42200155,2023-04-26 00:00:00,MEM vs. LAL,W,240,44.0,...,54.0,28.0,8.0,3.0,13.0,17.0,99.0,-17,1,Playoffs
65647,42022,1610612758,SAC,Sacramento Kings,42200165,2023-04-26 00:00:00,SAC vs. GSW,L,240,42.0,...,43.0,33.0,11.0,7.0,15.0,24.0,123.0,7,1,Playoffs
65648,42022,1610612749,MIL,Milwaukee Bucks,42200105,2023-04-26 00:00:00,MIL vs. MIA,L,265,42.0,...,45.0,25.0,6.0,5.0,11.0,31.0,128.0,2,1,Playoffs
65651,42022,1610612737,ATL,Atlanta Hawks,42200116,2023-04-27 00:00:00,ATL vs. BOS,L,240,44.0,...,47.0,24.0,7.0,10.0,7.0,16.0,128.0,8,1,Playoffs
65652,42022,1610612744,GSW,Golden State Warriors,42200166,2023-04-28 00:00:00,GSW vs. SAC,L,240,32.0,...,53.0,25.0,10.0,8.0,20.0,26.0,118.0,19,1,Playoffs
65653,42022,1610612747,LAL,Los Angeles Lakers,42200156,2023-04-28 00:00:00,LAL vs. MEM,W,240,49.0,...,45.0,19.0,10.0,2.0,14.0,17.0,85.0,-40,1,Playoffs
65654,42022,1610612743,DEN,Denver Nuggets,42200221,2023-04-29 00:00:00,DEN vs. PHX,W,240,48.0,...,38.0,20.0,4.0,5.0,16.0,19.0,107.0,-18,1,Playoffs
65655,42022,1610612752,NYK,New York Knicks,42200201,2023-04-30 00:00:00,NYK vs. MIA,L,240,41.0,...,39.0,26.0,8.0,5.0,9.0,14.0,108.0,7,1,Playoffs
